In [1]:
import gurobi as gb
from gurobipy import GRB

import networkx as nx
import numpy as np
import math

# Network model

In [2]:
from functions_library import *

# Gurobi Optimization
Implementation of the LP, which Arash proposed on 6. June 2022

In [3]:
opt_mod = gb.Model(name="linear programm")

## Setup Network Parameters

In [4]:
k = 4
G = init_ring(2**k)
D = init_uniformDemand_matrix_symmetric(G)
#D[0][15] = 1
#adjM = np.full((2**k,2**k),0)
#A = nx.adjacency_matrix(G)

## Setting variables

In [5]:
n = len(G.nodes)

allShortestPath = nx.shortest_path(G)

e = opt_mod.addMVar((len(G.nodes), len(G.nodes)), name="e", vtype="B")
dist = opt_mod.addMVar((len(G.nodes), len(G.nodes)), name="dist", vtype="I", lb=0, ub=GRB.INFINITY)
x = opt_mod.addMVar((len(G.nodes), len(G.nodes),len(G.nodes), len(G.nodes)), name="x", vtype="B")
opt_mod.update() 

## Adding constraints

In [6]:
# Adjacency Matrix (e) Constraints
opt_mod.addConstrs((e[i,(i+1) % len(G.nodes)] == 1 for i in G.nodes), name="c-ring")
opt_mod.addConstrs((e[i,i] == 0 for i in G.nodes), name="c-noSelfEdge")
opt_mod.addConstrs((e[i,j] == e[j,i] for i in G.nodes for j in G.nodes), name="c-undirected")

maxNumberE =  k+1 #math.log(n,2)-1 + 2
opt_mod.addConstrs((e[i,:].sum() <= maxNumberE for i in G.nodes), name="c-logE")


    
# Distance Matrix (dist) Constraints. dist[i][j]:= SP-length between i and j
opt_mod.addConstrs((x[i,j,:,:].sum()/2==dist[i,j] for i in G.nodes for j in G.nodes), name="c-subpath0")
opt_mod.addConstrs((dist[i,j] <= dist[i,u] + dist[u,j] for i in G.nodes for j in G.nodes for u in G.nodes),
                  name="c-dist2")




# Ensuring that x shows a correct path
opt_mod.addConstrs((x[i,j,u,v] <= e[u,v] for i in G.nodes for j in G.nodes for u in G.nodes for v in G.nodes),
                  name="flow0")
for i in G.nodes:
    for j in G.nodes:
        if i != j:
            opt_mod.addConstr(x[i,j,i,:].sum() == 1, name="c-flowStart")
            opt_mod.addConstr(x[i,j,:,j].sum() == 1, name="c-flowEnd")
            for v in G.nodes:
                if v not in [i,j]:
                    opt_mod.addConstr((x[i,j,:,v].sum() - x[i,j,v,:].sum()) == 0, name="blue0")

## Objective

In [ ]:
#print(len(x[0,1,x,y]))
nodesPairListNoDuplication = complete_node_pair_list_noDuplication(G)
opt_mod.setObjective(sum([dist[i,j]*D[i][j] for (i,j) in nodesPairListNoDuplication]), GRB.MINIMIZE)
opt_mod.update()


# Run
#opt_mod.display()
opt_mod.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 74032 rows, 66048 columns and 317408 nonzeros
Model fingerprint: 0x9aee67a6
Variable types: 0 continuous, 66048 integer (65792 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [7e-02, 7e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 13072 rows and 4744 columns
Presolve time: 0.63s
Presolved: 60960 rows, 61304 columns, 283104 nonzeros
Variable types: 0 continuous, 61304 integer (61032 binary)


In [ ]:
"""if opt_mod.status == GRB.Status.OPTIMAL:
    print('Obj Function:', opt_mod.objVal)
    for v in opt_mod.getVars():
        print(v.varName, v.x)       
else:
    print(opt_mod.status)"""

# Putting it back into a graph

In [ ]:
G_sol = nx.Graph()
for i in range(n): 
    G_sol.add_node(i)

for i in G_sol.nodes:
    for j in range(i+1,len(G_sol.nodes)):
        if e[i,j].X == 1.0:
            G_sol.add_edge(i,j)


In [ ]:
pos = nx.circular_layout(G_sol)
nx.draw_networkx(G_sol,pos=pos,with_labels=True)

In [ ]:
#print(sum([x[i,j,:,:].sum()*D[i][j] for (i,j) in complete_node_pair_list0]))

In [ ]:
print(calc_cost(G_sol,D))